In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json
from bertopic import BERTopic
from hdbscan import HDBSCAN
import re

ModuleNotFoundError: No module named 'pandas'

In [2]:
json_open = open("nogizaka_tweets.json","r",encoding="utf-8")
json_open = json.load(json_open)

id = []
text = []
viewCount = []
likeCount = []
url = []

for i in range(len(json_open)):
    id.append(json_open[i]["id"])
    text.append(json_open[i]["text"])
    viewCount.append(json_open[i]["viewCount"])
    likeCount.append(json_open[i]["likeCount"])
    url.append(json_open[i]["url"])

df = pd.DataFrame({
    "id":id,
    "text":text,
    "viewCount":viewCount,
    "likeCount":likeCount,
    "url":url})

df

,id,text,viewCount,likeCount,url
0,1970330163209789748,＼絶賛発売中‼️／\n#GoodsPress 10月号\n\n連載一周年を記念してリバーシブ...,50307,2305,https://x.com/GoodsPress/status/19703301632097...
1,1967592475046097140,色んな想いの人がいると思うけど、自分はアイドルが好きなんじゃなくて乃木坂46が大好き htt...,243350,4181,https://x.com/ikuma_nogi/status/19675924750460...
2,1969265264924532742,乃木坂46×So-netコラボキャンペーンサイトでは、CM撮影時のオフショットやメンバーから...,205970,6042,https://x.com/Sonet_fukuonsei/status/196926526...
3,1968646188762316907,次回の #乃木坂あそぶだけ は...\n\nNOGIZAKA ROOM SHARE 🏠\n*...,568518,6838,https://x.com/nogidoga46/status/19686461887623...
4,1970146155725312331,【今日は何の日】9月23日の乃木坂46\nhttps://t.co/xItIxpc5HY\n...,895,22,https://x.com/nogizaka46or/status/197014615572...
...,...,...,...,...,...
195,1969421340508897381,【今日は何の日】9月21日の乃木坂46\nhttps://t.co/SMjoikte3I\n...,688,24,https://x.com/nogidic3/status/1969421340508897381
196,1968347566548148699,【今日は何の日】9月18日の乃木坂46\nhttps://t.co/VR2W0pyOge\n...,6963,227,https://x.com/asukabiiki/status/19683475665481...
197,1966857129631080478,"#過剰接続 \nnoteメンバーシップ限定音声\n｢Tyler, The Creator来日...",32058,11,https://x.com/kimu_ra10/status/196685712963108...
198,1966807568992661877,『6th Anniversary 乃木坂46時間TV』を忘れない画像\nhttps://t....,3685,199,https://x.com/nogizaka46or/status/196680756899...


In [3]:
df["text"] = df["text"].str.replace("\n","")
pattern = r'https?://[^\s]+'
df["text"] = df["text"].apply(lambda x:re.sub(pattern,"",x))

In [6]:
# load model & clustering
hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples=3, metric='euclidean', cluster_selection_method='eom')

model = BERTopic(embedding_model="paraphrase-multilingual-MiniLM-L12-v2",
                hdbscan_model=hdbscan_model)  # 多言語モデルで日本語を使う
topics, probs = model.fit_transform(df["text"])
df_topic = model.get_topic_info()
df_topic

,Topic,Count,Name,Representation,Representative_Docs
0,-1,25,-1_so_bayfm_乃木坂46_co,"[so, bayfm, 乃木坂46, co, yuna, ゆなtime, このあと23, j...",[／「乃木坂46」×「So-net」本日9/22よりコラボTV CMを公開＼乃木坂46のパフ...
1,0,38,0_今日は何の日_遠藤さくら_9月18日の乃木坂46_賀喜遥香,"[今日は何の日, 遠藤さくら, 9月18日の乃木坂46, 賀喜遥香, 新参者, 9月23日の...",[【今日は何の日】9月19日の乃木坂46 結成10周年記念号」発売記念SHOWROOM生配信...
2,1,36,1_乃木坂46_乃木坂スター誕生_5期生_購入はこちら,"[乃木坂46, 乃木坂スター誕生, 5期生, 購入はこちら, 井上和, 川﨑桜, 梅澤美波,...",[【乃木坂46💜5期生】 『 銭湯ラプソディー 』菅原・川﨑・冨里・岡本・池田※セーラームー...
3,2,21,2_白石麻衣_西野七瀬_今日は何の日_乃木坂工事中,"[白石麻衣, 西野七瀬, 今日は何の日, 乃木坂工事中, 乃木のの, 乃木坂46, ガールズ...",[【乃木坂46💜バナナマン】 乃木坂工事中 ♪コロッケ兄弟😆２万のバイクの歌😅#白石麻衣...
4,3,15,3_ブログ更新_乃木坂の夏_2025_五百城茉央,"[ブログ更新, 乃木坂の夏, 2025, 五百城茉央, 2025年, 2024年, おまけ,...",[【呪術廻戦 × WIND AND SEA】コラボコレクションが国内4月17日に発売予定。キ...
5,4,13,4_ブログ更新_櫻坂46_6期生リレー_中西アルノ,"[ブログ更新, 櫻坂46, 6期生リレー, 中西アルノ, 1歳, 10月号にて, 10月号は...",[🅰️質問答えます！ RT🎁1️⃣0️⃣0️⃣0️⃣たまに不思議なお客さんが多いともお聞きい...
6,5,11,5_久保史緒里_rp_卒業を発表_久保史緒里卒業,"[久保史緒里, rp, 卒業を発表, 久保史緒里卒業, 乃木坂46久保史緒里, 乃木坂逃避行...",[乃木坂46・久保史緒里、卒業を発表 11月に横浜アリーナで卒業コンサート(オリコン) アイ...
7,6,10,6_ブログ更新_6期生リレー_鈴木佑捺_池田瑛紗,"[ブログ更新, 6期生リレー, 鈴木佑捺, 池田瑛紗, 冨里奈央, 19歳, 今日の乃木坂4...","[【ブログ更新 6期生リレー】 新参者 川端晃菜 , 【ブログ更新 6期生リレー】 おさげ..."
8,7,8,7_1760円_①上瞼に薄いブラウン_①目尻_①肌馴染みの良いオレンジ系のチークを丸く頬に乗せ,"[1760円, ①上瞼に薄いブラウン, ①目尻, ①肌馴染みの良いオレンジ系のチークを丸く頬...",[乃木坂46の伊藤理々杏ちゃんが愛用してるキングダムの束感マスカラ良すぎて優勝🥲束感まつ毛で...
9,8,6,8_music_mvで表題曲を選ぶ中_mステで久保ちゃんの代打を林が努めた大切な一曲卒業コン...,"[music, mvで表題曲を選ぶ中, mステで久保ちゃんの代打を林が努めた大切な一曲卒業コ...",[多くのメンバーが乃木坂46の好きな曲•MVで表題曲を選ぶ中「急斜面」と「立ち直り中」を選曲...


In [9]:
df_topic_doc = model.get_document_info(df["text"])
df_topic_doc

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,＼絶賛発売中‼️／#GoodsPress 10月号連載一周年を記念してリバーシブル表紙に登場...,0,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月,"[乃木坂46, 今日は何の日, 乃木坂スター誕生, 菅原咲月, 5期生, 井上和, 白石麻衣...",[【乃木坂46💜5期生】新・乃木坂スター誕生！『 金八先生から 贈る言葉 』#乃木坂46 #...,乃木坂46 - 今日は何の日 - 乃木坂スター誕生 - 菅原咲月 - 5期生 - 井上和 -...,0.925885,False
1,色んな想いの人がいると思うけど、自分はアイドルが好きなんじゃなくて乃木坂46が大好き,1,1_ブログ更新_久保史緒里_6期生リレー_乃木坂46,"[ブログ更新, 久保史緒里, 6期生リレー, 乃木坂46, 瀬戸口心月, rp, みたいな,...","[【ブログ更新 久保史緒里】 光のような、夏でした , 【ブログ更新 久保史緒里】 風のよ...",ブログ更新 - 久保史緒里 - 6期生リレー - 乃木坂46 - 瀬戸口心月 - rp - ...,1.000000,False
2,乃木坂46×So-netコラボキャンペーンサイトでは、CM撮影時のオフショットやメンバーから...,-1,-1_乃木坂46_so_いいね_賀喜遥香,"[乃木坂46, so, いいね, 賀喜遥香, 元乃木坂46, blu, ray, 与田祐希,...",[／「乃木坂46」×「So-net」本日9/22よりコラボTV CMを公開＼乃木坂46のパフ...,乃木坂46 - so - いいね - 賀喜遥香 - 元乃木坂46 - blu - ray -...,0.000000,False
3,次回の #乃木坂あそぶだけ は...NOGIZAKA ROOM SHARE 🏠** ┈┈┈┈...,0,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月,"[乃木坂46, 今日は何の日, 乃木坂スター誕生, 菅原咲月, 5期生, 井上和, 白石麻衣...",[【乃木坂46💜5期生】新・乃木坂スター誕生！『 金八先生から 贈る言葉 』#乃木坂46 #...,乃木坂46 - 今日は何の日 - 乃木坂スター誕生 - 菅原咲月 - 5期生 - 井上和 -...,1.000000,False
4,【今日は何の日】9月23日の乃木坂46,0,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月,"[乃木坂46, 今日は何の日, 乃木坂スター誕生, 菅原咲月, 5期生, 井上和, 白石麻衣...",[【乃木坂46💜5期生】新・乃木坂スター誕生！『 金八先生から 贈る言葉 』#乃木坂46 #...,乃木坂46 - 今日は何の日 - 乃木坂スター誕生 - 菅原咲月 - 5期生 - 井上和 -...,1.000000,False
...,...,...,...,...,...,...,...,...
195,【今日は何の日】9月21日の乃木坂46,0,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月,"[乃木坂46, 今日は何の日, 乃木坂スター誕生, 菅原咲月, 5期生, 井上和, 白石麻衣...",[【乃木坂46💜5期生】新・乃木坂スター誕生！『 金八先生から 贈る言葉 』#乃木坂46 #...,乃木坂46 - 今日は何の日 - 乃木坂スター誕生 - 菅原咲月 - 5期生 - 井上和 -...,1.000000,False
196,【今日は何の日】9月18日の乃木坂46 知ってて当然!!! ボーダークイズ 後半戦』#齋藤飛鳥,0,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月,"[乃木坂46, 今日は何の日, 乃木坂スター誕生, 菅原咲月, 5期生, 井上和, 白石麻衣...",[【乃木坂46💜5期生】新・乃木坂スター誕生！『 金八先生から 贈る言葉 』#乃木坂46 #...,乃木坂46 - 今日は何の日 - 乃木坂スター誕生 - 菅原咲月 - 5期生 - 井上和 -...,1.000000,False
197,"#過剰接続 noteメンバーシップ限定音声｢Tyler, The Creator来日公演と乃...",0,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月,"[乃木坂46, 今日は何の日, 乃木坂スター誕生, 菅原咲月, 5期生, 井上和, 白石麻衣...",[【乃木坂46💜5期生】新・乃木坂スター誕生！『 金八先生から 贈る言葉 』#乃木坂46 #...,乃木坂46 - 今日は何の日 - 乃木坂スター誕生 - 菅原咲月 - 5期生 - 井上和 -...,1.000000,False
198,『6th Anniversary 乃木坂46時間TV』を忘れない画像,0,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月,"[乃木坂46, 今日は何の日, 乃木坂スター誕生, 菅原咲月, 5期生, 井上和, 白石麻衣...",[【乃木坂46💜5期生】新・乃木坂スター誕生！『 金八先生から 贈る言葉 』#乃木坂46 #...,乃木坂46 - 今日は何の日 - 乃木坂スター誕生 - 菅原咲月 - 5期生 - 井上和 -...,0.928823,False


In [ ]:
summaries = []
for _,row in grouped.iterrows():
    topic_id = row["Topic"]
    docs = row["Document"]

    input_text = "\n".join(docs[:10]) 

    

In [7]:
from bertopic import BERTopic
from hdbscan import HDBSCAN

# BERTopic の学習
model = BERTopic(embedding_model="paraphrase-multilingual-MiniLM-L12-v2")
topics, probs = model.fit_transform(df["text"])

# df にトピック情報を追加
df["topic"] = topics
df["probability"] = probs  # 各文がクラスタに属する確率

# トピックごとの代表語を付与
topic_info = model.get_topic_info()
topic_repr = {row["Topic"]: row["Name"] for _, row in topic_info.iterrows()}
df["topic_repr"] = df["topic"].map(topic_repr)

df.head()


,id,text,viewCount,likeCount,url,topic,probability,topic_repr
0,1970330163209789748,＼絶賛発売中‼️／#GoodsPress 10月号連載一周年を記念してリバーシブル表紙に登場...,50307,2305,https://x.com/GoodsPress/status/19703301632097...,0,0.925885,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月
1,1967592475046097140,色んな想いの人がいると思うけど、自分はアイドルが好きなんじゃなくて乃木坂46が大好き,243350,4181,https://x.com/ikuma_nogi/status/19675924750460...,1,1.000000,1_ブログ更新_久保史緒里_6期生リレー_乃木坂46
2,1969265264924532742,乃木坂46×So-netコラボキャンペーンサイトでは、CM撮影時のオフショットやメンバーから...,205970,6042,https://x.com/Sonet_fukuonsei/status/196926526...,-1,0.000000,-1_乃木坂46_so_いいね_賀喜遥香
3,1968646188762316907,次回の #乃木坂あそぶだけ は...NOGIZAKA ROOM SHARE 🏠** ┈┈┈┈...,568518,6838,https://x.com/nogidoga46/status/19686461887623...,0,1.000000,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月
4,1970146155725312331,【今日は何の日】9月23日の乃木坂46,895,22,https://x.com/nogizaka46or/status/197014615572...,0,1.000000,0_乃木坂46_今日は何の日_乃木坂スター誕生_菅原咲月


### やること
<li>どのEmbeddingモデルが適しているのか調査</li>
<li>前処理（URLの削除、/nの削除）</li>

## 前処理の実験

In [21]:
import re
df["text"] = df["text"].str.replace("\n","")
pattern = r'https?://[^\s]+'
df["text"] = df["text"].apply(lambda x:re.sub(pattern,"",x))
df["text"].to_csv("before_preprocess.csv")
df["text"]

0      ＼絶賛発売中‼️／#GoodsPress 10月号連載一周年を記念してリバーシブル表紙に登場...
1            色んな想いの人がいると思うけど、自分はアイドルが好きなんじゃなくて乃木坂46が大好き 
2      乃木坂46×So-netコラボキャンペーンサイトでは、CM撮影時のオフショットやメンバーから...
3      次回の #乃木坂あそぶだけ は...NOGIZAKA ROOM SHARE 🏠** ┈┈┈┈...
4                                   【今日は何の日】9月23日の乃木坂46 
                             ...                        
195                                 【今日は何の日】9月21日の乃木坂46 
196     【今日は何の日】9月18日の乃木坂46 知ってて当然!!! ボーダークイズ 後半戦』#齋藤飛鳥 
197    #過剰接続 noteメンバーシップ限定音声｢Tyler, The Creator来日公演と乃...
198                  『6th Anniversary 乃木坂46時間TV』を忘れない画像 
199    (このような番組を作ってくれたことに大感謝なのは前提として)わざわざこんな遠出しなくても「ま...
Name: text, Length: 200, dtype: object

## サンプルコード

In [51]:
# 1. Load a pretrained Sentence Transformer model
# 日本語の学習済みモデル
model = SentenceTransformer("colorfulscoop/sbert-base-ja")

# The sentences to encode
sample_sentences = [
    "乃木坂46の新曲が発売されました",
    "日本の夏は非常に暑いです",
    "アイドルの握手会を私は毎回楽しみにしている",
]

# 2. Calculate embeddings by calling model.encode()
sample_embeddings = model.encode(sample_sentences)
print(sample_embeddings.shape)

# 3. Calculate the embedding similarities
sample_similarities = model.similarity(sample_embeddings, sample_embeddings)
print(sample_similarities)

(3, 768)
tensor([[1.0000, 0.2829, 0.5565],
        [0.2829, 1.0000, 0.2934],
        [0.5565, 0.2934, 1.0000]])


## Embeddingの実験

実験結果：https://docs.google.com/spreadsheets/d/19FwW4YFM1r99dX8_mzFjqXaRZRPDIeYpdZagtp3DIg8/edit?gid=0#gid=0

In [11]:
model = SentenceTransformer("colorfulscoop/sbert-base-ja")
embeddings = model.encode(df["text"].tolist())
df["embeddings"] = list(embeddings)

"""
特徴

Embeddingを自前で作る
→ SentenceTransformer("colorfulscoop/sbert-base-ja") で日本語特化モデルを使える

BERTopic に渡す時点では 既に埋め込みがあるので、BERTopic 内部で Embedding は作られない

自由にモデルを選べる（日本語特化、最新モデルなど）

大規模データの場合、先に埋め込みを作ってキャッシュできるので効率的

"""

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(df["text"].to_list(), embeddings)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,173,0_46_2025_92346_10,"[46, 2025, 92346, 10, 91846, 46ann, 2018, http...",[／ \n📢情報解禁 第1弾\n2年ぶりに開催決定✨\n「新参者 二〇二五 in TOKYU...
1,1,27,1_108_19_bluray_21,"[108, 19, bluray, 21, httpstco0pzvlvetlj, http...",[【ブログ更新 中西アルノ】 あなたが笑っていたらそれで https://t.co/pd6S...


In [13]:
# load model & clustering
model = BERTopic(embedding_model="paraphrase-multilingual-MiniLM-L12-v2")  # 多言語モデルで日本語を使う

"""
特徴

BERTopic が内部で自動的に埋め込みを作る
→ embedding_model に指定したモデル（ここでは多言語 MiniLM）を使う

コードは簡潔で書きやすい

埋め込みのキャッシュや細かい設定はやりにくい

利用できるモデルは Hugging Face 上で sentence-transformers 互換のものに限られる

"""
topics, probs = model.fit_transform(df["text"])
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,119,-1_co_https_乃木坂46_今日は何の日,"[co, https, 乃木坂46, 今日は何の日, 久保史緒里, 遠藤さくら, 齋藤飛鳥,...",[【乃木坂46】賀喜遥香ちゃん\n気に入ったらRT＆いいね!!\n#賀喜遥香 #乃木坂46 ...
1,0,36,0_乃木坂46_https_co_乃木坂スター誕生,"[乃木坂46, https, co, 乃木坂スター誕生, 5期生, 新参者, 井上和, 菅原...",[【乃木坂46💜5期生】\n新・乃木坂スター誕生！\n\n『 金八先生から 贈る言葉 』\n...
2,1,17,1_https_co_ブログ更新_6期生リレー,"[https, co, ブログ更新, 6期生リレー, 全公演グッズ再販, ueimpgyqt...",[【ブログ更新 6期生リレー】 新参者 川端晃菜 https://t.co/iTT0Tson...
3,2,16,2_https_co_ブログ更新_みたいな,"[https, co, ブログ更新, みたいな, 深川麻衣, 久保史緒里, 6期生リレー, ...",[【ブログ更新 6期生リレー】 もしあなたが急に世界からいなくなったら 矢田萌華 https...
4,3,12,3_https_co_今日は何の日_西野七瀬,"[https, co, 今日は何の日, 西野七瀬, 2017年, 生田絵梨花, 乃木坂工事中...",[JO1、テレ朝「ドリフェス」出演決定❗️\nhttps://t.co/bH71mEKnLD...


In [12]:
# load model & clustering
model = BERTopic(embedding_model="Hanpt/sentence-transformer-ja-triplet")
#特徴：Triplet Lossで学習された日本語向けの埋め込みモデルで、類似度計算に優れています。
#用途：文書の類似度計算、クラスタリングなど。
#時間かかる

topics, probs = model.fit_transform(df["text"])
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,20,-1_https_co_乃木坂46_5期生,"[https, co, 乃木坂46, 5期生, 池田瑛紗, セーラームーン, ムーンライト伝...",[【乃木坂46💜5期生】\n\n 『 銭湯ラプソディー 』\n菅原・川﨑・冨里・岡本・池田\...
1,0,134,0_co_https_乃木坂46_今日は何の日,"[co, https, 乃木坂46, 今日は何の日, 久保史緒里, ブログ更新, 井上和, ...",[本日9月23日(火)23:00から、\nBAYFM「乃木坂46 柴田柚菜のDreaming...
2,1,27,1_https_co_乃木坂46_購入はこちら,"[https, co, 乃木坂46, 購入はこちら, リアルミーグリ, 昨日のリアミにて着用...","[💜乃木坂46 梅澤美波ちゃん着用\n\n■ TOD’S\nレザーバレリーナ\n¥116,6..."
3,2,19,2_https_co_goodspress_乃木坂46,"[https, co, goodspress, 乃木坂46, naographic, 冨里奈...",[ぐぬぬ…(卒倒)\n\n#与田祐希 #五百城茉央 #菅原咲月\n\n(via 乃木坂46 ...


In [3]:
# load model & clustering
model = BERTopic(embedding_model="sonoisa/sentence-bert-base-ja-mean-tokens")

"""
🧠 モデル概要
モデル名: sonoisa/sentence-bert-base-ja-mean-tokens
タイプ: Sentence-BERT 日本語版
ベースモデル: cl-tohoku/bert-base-japanese
学習方法:
BERTで得たトークン埋め込みを mean pooling（平均化）して文ベクトルを作成
日本語テキストの意味類似度タスクに最適化

✅ 特徴
日本語特化:日本語Wikipediaや青空文庫など大規模日本語コーパスで事前学習。
文単位の意味表現に強い：「乃木坂ライブ最高！」と「乃木坂のコンサート楽しかった！」
→ 高い類似度を返す
単語レベルよりも 文全体の意味 をうまく捉えられる

用途に最適
・クラスタリング（BERTopic, HDBSCAN, KMeans）
・文類似度検索（semantic search）
・要約の前処理（近い文をまとめる）
"""

topics, probs = model.fit_transform(df["text"])
model.get_topic_info()

No sentence-transformers model found with name sonoisa/sentence-bert-base-ja-mean-tokens. Creating a new one with mean pooling.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


,Topic,Count,Name,Representation,Representative_Docs
0,-1,5,-1_6期生リレー_ブログ更新_https_co,"[6期生リレー, ブログ更新, https, co, 鈴木佑捺, bwcgjhzqti, 8...",[【ブログ更新 6期生リレー】 2025 乃木坂の夏 瀬戸口心月 https://t.co/...
1,0,163,0_co_https_乃木坂46_久保史緒里,"[co, https, 乃木坂46, 久保史緒里, 井上和, 乃木坂スター誕生, ブログ更新...",[#真夏の全国ツアー2025\n\n全公演グッズ再販\nhttps://t.co/UeiMp...
2,1,32,1_https_co_今日は何の日_白石麻衣,"[https, co, 今日は何の日, 白石麻衣, 9月23日の乃木坂46, 齋藤飛鳥, x...",[【今日は何の日】9月19日の乃木坂46\nhttps://t.co/2Zl3SlRREj\...


In [7]:
# load model & clustering
model = BERTopic(embedding_model="ku-nlp/deberta-v2-base-japanese")

"""
特徴: DeBERTaベースの日本語モデル。BERTより精度が高い。
得意分野: 分類・文表現獲得。
注意点: Sentence-BERTのように直接「埋め込み出力」には最適化されていないので、クラスタリング利用時はPooling処理が必要。
"""

topics, probs = model.fit_transform(df["text"])
model.get_topic_info()

No sentence-transformers model found with name ku-nlp/deberta-v2-base-japanese. Creating a new one with mean pooling.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


,Topic,Count,Name,Representation,Representative_Docs
0,-1,3,-1_rimless_glasses_980_épine,"[rimless, glasses, 980, épine, エピヌ, 池田瑛紗, mtwf...",[本日9月18日(木)19:00頃～、 #SHOWROOM 「#のぎおび」に #菅原咲月 が...
1,0,119,0_co_https_乃木坂46_久保史緒里,"[co, https, 乃木坂46, 久保史緒里, 11, 井上和, 新参者, リアミ, 菅...",[このあと23:00〜 #ゆなtime 💡\n\n今週はネコちゃんが気になる #柴田柚菜 へ...
2,1,31,1_https_co_ブログ更新_6期生リレー,"[https, co, ブログ更新, 6期生リレー, 久保史緒里, 五百城茉央, を忘れない...",[【ブログ更新 6期生リレー】 新参者 川端晃菜 https://t.co/iTT0Tson...
3,2,29,2_今日は何の日_https_co_白石麻衣,"[今日は何の日, https, co, 白石麻衣, 9月23日の乃木坂46, 9月18日の乃...",[【今日は何の日】9月23日の乃木坂46\nhttps://t.co/yUzxmrOIyF\...
4,3,18,3_乃木坂46_乃木坂スター誕生_5期生_冨里奈央,"[乃木坂46, 乃木坂スター誕生, 5期生, 冨里奈央, 気に入ったらrt, いいね, ht...",[【井上和💜中西アルノ】\n 超・乃木坂スター誕生！\n\n 『 Lemon 』米津玄師...


In [8]:
# load model & clustering
model = BERTopic(embedding_model="rinna/japanese-roberta-base")

"""
特徴: RoBERTaベースで大量日本語学習済み。
得意分野: 文意の把握に強い。
クラスタリング利用: sentence-transformersでfine-tuning済みモデルもあるので、相性良い。
"""

topics, probs = model.fit_transform(df["text"])
model.get_topic_info()

No sentence-transformers model found with name rinna/japanese-roberta-base. Creating a new one with mean pooling.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of RobertaModel were not initialized from the model checkpoint at rinna/japanese-roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, an

,Topic,Count,Name,Representation,Representative_Docs
0,0,112,0_co_https_乃木坂46_今日は何の日,"[co, https, 乃木坂46, 今日は何の日, 白石麻衣, 久保史緒里, ブログ更新,...",[【今日は何の日】9月19日の乃木坂46\nhttps://t.co/2Zl3SlRREj\...
1,1,75,1_co_https_乃木坂46_井上和,"[co, https, 乃木坂46, 井上和, 久保史緒里, 乃木のの, 菅原咲月, いいね...",[＜今夜の #乃木のの＞\n\n本日9/14(日) 18:00から\n#菅原咲月(MC)、#...
2,2,13,2_https_co_乃木坂46_naographic,"[https, co, 乃木坂46, naographic, goodspress, リアミ...",[乃木坂46 『Same numbers』リアミ\n３部 梅澤 美波\n\n３期生レーンも気...


In [6]:
# load model & clustering
model = BERTopic(embedding_model="cl-tohoku/bert-base-japanese-v3")

"""
特徴: 東北大の標準BERT。多くの研究・実装で利用。
得意分野: 文分類のベースライン。
活用方法: Sentence-Transformersに組み込んでsentence embedding化するとクラスタリング可能。
"""

topics, probs = model.fit_transform(df["text"])
model.get_topic_info()

No sentence-transformers model found with name cl-tohoku/bert-base-japanese-v3. Creating a new one with mean pooling.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


,Topic,Count,Name,Representation,Representative_Docs
0,0,166,0_co_https_乃木坂46_今日は何の日,"[co, https, 乃木坂46, 今日は何の日, 久保史緒里, 井上和, 菅原咲月, 白...",[【今日は何の日】9月18日の乃木坂46\nhttps://t.co/m981br1cEx\...
1,1,34,1_https_co_ブログ更新_6期生リレー,"[https, co, ブログ更新, 6期生リレー, 久保史緒里, 五百城茉央, 全公演グッ...",[【ブログ更新 6期生リレー】 もしあなたが急に世界からいなくなったら 矢田萌華 https...
